In [1]:
!pip install geemap

In [2]:
import hydrofunctions as hf
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import folium
import geemap.foliumap as geemap
import ee
import matplotlib.dates as mdates
import hydrofns

%matplotlib inline

In [3]:
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1AX4XfWhcWTYrSoGxX3tLu6hAFl5OmtazIYwSdeJWEGkRilWc0WpyqPHsdB0

Successfully saved authorization token.


In [18]:
# Site number for NWIS gage (Pine Bluff AR Site)
pineBluff = ['07263650']

# Coordinates for the bounds of a rectangle at Pine Bluff AR
pineBluff_coord1 = [-91.75, 34.1845]
pineBluff_coord2 = [-92.1, 34.42]
pineBluff_coord3 = [-92.1973, 34.3831]
pineBluff_coord4 = [-91.83, 34.12]
pineBluff_coord5 = [-91.6933, 34.1845]

# Generate Pine Bluff site bounds geometry
pineBluff_rectangleGeoJSON = ee.Geometry.Polygon([
    [pineBluff_coord1, pineBluff_coord2, pineBluff_coord3,
        pineBluff_coord4, pineBluff_coord5]
])
pineBluff_bnds = ee.FeatureCollection(ee.Feature(pineBluff_rectangleGeoJSON,
                                {'name': 'Pine Bluff, AR', 'fill': 1}))

# Palettes for visualizations
trueColor_palette = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}
waterPalette = ['red', 'yellow', 'green', 'blue']

# Import the Sentinel 2 collection
sentinel2 = ee.ImageCollection(
    'COPERNICUS/S2_SR').filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 5))

# Define wet and dry month dates for Pine Bluff
pineBluff_wet_season_start = '2019-06-01'
pineBluff_wet_season_end = '2019-06-30'

pineBluff_dry_season_start = '2019-01-01'
pineBluff_dry_season_end = '2019-01-30'

In [19]:
# Create images for wet and dry seasons at Pine Bluff AR
pineBluff_dry_season = sentinel2.filterBounds(pineBluff_bnds).filterDate(
    pineBluff_dry_season_start, pineBluff_dry_season_end).mosaic().clip(pineBluff_bnds)

pineBluff_wet_season = sentinel2.filterBounds(pineBluff_bnds).filterDate(
    pineBluff_wet_season_start, pineBluff_wet_season_end).mosaic().clip(pineBluff_bnds)

In [27]:
# Create true color dry season image for Pine Bluff AR
pb_true_color_dry = geemap.Map(center=[34.3, -92], zoom=11)
pb_true_color_dry.addLayer(pineBluff_dry_season, trueColor_palette,
             '2019 Dry season true color')
pb_true_color_dry.addLayerControl()
pb_true_color_dry

In [28]:
# Create true color wet season image for Pine Bluff AR
pb_true_color_wet = geemap.Map(center=[34.3, -92], zoom=11)
pb_true_color_wet.addLayer(pineBluff_wet_season, trueColor_palette,
             '2019 Wet season true color')
pb_true_color_wet.addLayerControl()
pb_true_color_wet

In [31]:
# Create NDWI dry season image for Pine Bluff AR
pineBluff_ndwi_map_dry = geemap.Map(center=[34.3, -92], zoom=11)
pineBluff_ndwi_dry = pineBluff_dry_season.normalizedDifference(['B3', 'B8'])
pineBluff_ndwi_map_dry.addLayer(pineBluff_ndwi_dry, {
    'min': -1, 'max': 0.5, 'palette': waterPalette}, '2019 Dry season NDWI')
pineBluff_ndwi_map_dry

In [32]:
# Create NDWI wet season image for Pine Bluff AR
pineBluff_ndwi_map_wet = geemap.Map(center=[34.3, -92], zoom=11)
pineBluff_ndwi_wet = pineBluff_wet_season.normalizedDifference(['B3', 'B8'])
pineBluff_ndwi_map_wet.addLayer(pineBluff_ndwi_wet, {
    'min': -1, 'max': 0.5, 'palette': waterPalette}, '2019 Dry season NDWI')
pineBluff_ndwi_map_wet

In [35]:
# Create dry season water map
pineBluff_water_map_dry = geemap.Map(center=[34.3, -92], zoom=11)

ndwi_threshold = 0
pineBluff_water_image_dry = pineBluff_ndwi_dry.gt(
    ndwi_threshold).selfMask()

pineBluff_water_map_dry.addLayer(pineBluff_water_image_dry, {
                                 'palette': 'blue'}, 'Water image')
pineBluff_water_map_dry

In [36]:
# Create dry season water map
pineBluff_water_map_wet = geemap.Map(center=[34.3, -92], zoom=11)

ndwi_threshold = 0
pineBluff_water_image_wet = pineBluff_ndwi_wet.gt(
    ndwi_threshold).selfMask()

pineBluff_water_map_wet.addLayer(pineBluff_water_image_wet, {
                                 'palette': 'blue'}, 'Water image')
pineBluff_water_map_wet